# Max-SAT: LP Rounding versus CP-SAT

In [323]:
# Optional[A] means either type A or None
# Union[A, B] means either type A or type B
from typing import Union, Optional, List, Tuple
Lit = int
Var = int
Clause = List[Lit]
CNF = List[Clause]
# A (partial) assignment is an array storing True/False/None for each variable
Assignment = List[Optional[bool]]

In [324]:
from random import shuffle, randint
import numpy as np

# random k-CNFs with n variables and m clauses
def F(k: int, n: int, m: int) -> CNF:
    # Fill in here
    vars = [i + 1 for i in range(n)]
    cnf = []
    for _ in range(m):
        shuffle(vars)
        # coin flip (-1 or 1)
        flips = [randint(0, 1) * 2 - 1 for _ in range(k)]
        cnf.append(list(np.multiply(vars[:k], flips)))
    return cnf

In [325]:
def count_sat(CNF, asses):
    def check_lit(lit):
        var = abs(lit)
        sign = lit // var 
        return asses[var] == sign
    def clause_sated(clause):
        filtered = list(filter(check_lit, clause))
        return len(filtered) > 0
    sated = list(filter(clause_sated, CNF))
    return len(sated)

Helper stuff out of the way, we'll start off with the LP rounding solution

In [326]:
# LP implementation
from ortools.linear_solver import pywraplp

def solveLP(CNF, k, n, m):
    solver = pywraplp.Solver("lp", pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    zs = [solver.NumVar(0, 1, f'z_{j}') for j in range(m)] # variable for if the j-th clause is satisfied
    ys = [solver.NumVar(0, 1, f'y_{i}') for i in range(n+1)] # variable for if the i-th clause is set to True
    # add constraints
    for j in range(m):
        clause = CNF[j]
        # count number of negatives
        neg_count = sum(1 for lit in clause if lit < 0)
        constraint = solver.Constraint(-neg_count, solver.infinity()) # must be larger than z_j
        constraint.SetCoefficient(zs[j], -1)
        for lit in clause:
            var = abs(lit)
            sign = lit / var
            constraint.SetCoefficient(ys[var], sign)
    # maximize number of satisfied clauses
    objective = solver.Objective()
    for zj in zs:
        objective.SetCoefficient(zj, 1)
    objective.SetMaximization()
    solver.Solve()
    def get_sols(xs):
        return list(map(lambda x : x.solution_value(), xs))
    return (get_sols(ys), get_sols(zs))
    return solver

In [327]:
# now, we do the rounding
from random import choices

# Johnson's rounding scheme: set each var with probability 1/2
def johnson(n):
    return [randint(0, 1) * 2 - 1 for _ in range(n + 1)]

# LP rounding scheme
def lp_rounding(CNF, ys):
    return [choices([1, -1], cum_weights=[ys[i], 1]) for i in range(len(ys))]

# solve the LP, do both rounding schemes, output the better result
def do_lp_algo(CNF, k, n, m):
    ys, zs = solveLP(CNF, k, n, m)

    ass1 = johnson(n)
    ass2 = lp_rounding(CNF, ys)
    print(ass1)

    count1 = count_sat(CNF, ass1)
    count2 = count_sat(CNF, ass2)

    return max(count1, count2)

In [328]:
k = 3
n = 3
m = 50
ex = F(k, n, m)
do_lp_algo(ex, k, n, m)

[-1, 1, -1, -1]


47

Now, we'll do the ILP rounding. This is essentially the same as the LP rounding model, except now we use CP-SAT rather than Glop

In [329]:
# ILP version
from ortools.sat.python import cp_model

def solveILP(CNF, k, n, m):
    model = cp_model.CpModel()
    zs = [model.NewIntVar(0, 1, f'z_{j}') for j in range(m)] # variable for if the j-th clause is satisfied
    ys = [model.NewIntVar(0, 1, f'y_{i}') for i in range(n+1)] # variable for if the i-th clause is set to True
    # add constraints
    for j in range(m):
        clause = CNF[j]
        # count number of negatives
        # neg_count = sum(1 for lit in clause if lit < 0)
        # constraint = solver.Constraint(-neg_count, solver.infinity()) # must be larger than z_j
        # constraint.SetCoefficient(zs[j], -1)
        lhs = 0
        for lit in clause:
            var = abs(lit)
            sign = lit / var
            if sign == 1:
                lhs += ys[var]
            else:
                lhs += 1 - ys[var]
        model.Add(lhs >= zs[j])
    # maximize number of satisfied clauses
    model.Maximize(sum(zs))

    solver = cp_model.CpSolver()
    solver.Solve(model)
    def get_sols(xs):
        return list(map(lambda x : solver.Value(x), xs))
    return (get_sols(ys), get_sols(zs))
    return solver

In [330]:
ys, zs = solveILP(ex, k, n, m)

In [331]:
ys

[0, 1, 0, 1]

In [332]:
zs

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [333]:
ex

[[2, 1, -3],
 [2, 1, 3],
 [-2, -3, -1],
 [1, -3, -2],
 [1, 3, -2],
 [1, -2, 3],
 [-1, -2, 3],
 [3, 1, 2],
 [2, -1, 3],
 [-2, 3, 1],
 [1, -3, -2],
 [-2, 1, 3],
 [-1, 3, -2],
 [1, 2, 3],
 [-2, -1, -3],
 [3, 1, -2],
 [-2, 3, 1],
 [1, 2, -3],
 [1, -2, 3],
 [-1, 3, 2],
 [1, -2, -3],
 [3, 2, -1],
 [-2, -1, 3],
 [3, 2, 1],
 [-1, 3, 2],
 [1, -2, -3],
 [-2, 1, 3],
 [3, -1, -2],
 [3, 1, -2],
 [1, -2, 3],
 [3, -1, 2],
 [-2, 3, 1],
 [-1, 2, -3],
 [1, 2, 3],
 [-3, 1, 2],
 [2, -1, 3],
 [1, 3, 2],
 [3, 1, -2],
 [-2, -3, -1],
 [3, -1, -2],
 [3, -2, 1],
 [-3, -2, -1],
 [2, 3, 1],
 [-2, -3, 1],
 [3, -2, 1],
 [1, -2, -3],
 [-2, 3, 1],
 [2, 1, 3],
 [3, 2, -1],
 [2, 1, 3]]